# Module 1 Homework
## Docker & SQL

In this homework we'll prepare the environment and practice with Docker and SQL

Question 1. Knowing docker tags

Run the command to get information on Docker

docker --help

Now run the command to get help on the "docker build" command:

docker build --help

Do the same for "docker run".

Which tag has the following text? - Automatically remove the container when it exits

    --delete
    --rc
    --rmc
    --rm

In [1]:
# sudo docker --help

Question 2. Understanding docker first run

Run docker with the python:3.9 image in an interactive mode and the entrypoint of bash. Now check the python modules that are installed ( use pip list ).

What is version of the package wheel ?

    0.42.0
    1.0.0
    23.0.1
    58.1.0

In [2]:
# sudo docker run -it puthon:3.9 --entrypoint bash
# pip list
# 0.40.0

Prepare Postgres

Run Postgres and load data as shown in the videos We'll use the green taxi trips from September 2019:
<code>
wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-09.csv.gz
</code>

You will also need the dataset with zones:
<code>
wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
</code>

Download this data and put it into Postgres (with jupyter notebooks or with a pipeline)

In [3]:
import pandas as pd
from sqlalchemy import create_engine

In [4]:
user='root'
password='root'
host='localhost'
port=5432
dbname='ny_taxi'
engine = create_engine(f"postgresql://{user}:{password}@{host}/{dbname}")

Question 3. Count records

How many taxi trips were totally made on September 18th 2019?

Tip: started and finished on 2019-09-18.

Remember that lpep_pickup_datetime and lpep_dropoff_datetime columns are in the format timestamp (date and hour+min+sec) and not in date.

    15767
    15612
    15859
    89009

In [5]:
pd.read_sql_query(sql="""SELECT COUNT(1) AS NUM_TRIPS FROM green_trip_data
WHERE DATE(lpep_pickup_datetime) = TO_DATE('2019-09-18','YYYY-MM-DD')
AND DATE(lpep_dropoff_datetime) = TO_DATE('2019-09-18','YYYY-MM-DD');""", con=engine)

,num_trips
0,15612


Question 4. Largest trip for each day

Which was the pick up day with the largest trip distance. Use the pick up time for your calculations.

    2019-09-18
    2019-09-16
    2019-09-26
    2019-09-21

In [6]:
pd.read_sql_query(sql="""SELECT DATE(lpep_pickup_datetime)
FROM green_trip_data
ORDER BY EXTRACT(EPOCH FROM (lpep_dropoff_datetime - Lpep_pickup_datetime)) DESC
LIMIT 1;""", con=engine)

,date
0,2019-09-26


Question 5. The number of passengers

Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?

    "Brooklyn" "Manhattan" "Queens"
    "Bronx" "Brooklyn" "Manhattan"
    "Bronx" "Manhattan" "Queens"
    "Brooklyn" "Queens" "Staten Island"

In [7]:
pd.read_sql_query(sql="""SELECT b."Borough" --SUM(a.total_amount) 
FROM green_trip_data as a
INNER JOIN taxi_zone as b ON a."PULocationID" = b."LocationID"
GROUP BY b."Borough"
HAVING SUM(a.total_amount) >5000
ORDER BY SUM(a.total_amount) DESC
LIMIT 3;""", con=engine)

,Borough
0,Brooklyn
1,Queens
2,Manhattan


Question 6. Largest tip

For the passengers picked up in September 2019 in the zone name Astoria which was the drop off zone that had the largest tip? We want the name of the zone, not the id.

Note: it's not a typo, it's tip , not trip

    Central Park
    Jamaica
    JFK Airport
    Long Island City/Queens Plaza

In [8]:
pd.read_sql_query(sql="""SELECT c."Zone" 
FROM green_trip_data as a
INNER JOIN taxi_zone as b ON a."PULocationID" = b."LocationID"
INNER JOIN taxi_zone as c ON a."DOLocationID" = c."LocationID"
WHERE EXTRACT('YEAR' FROM a.lpep_pickup_datetime) = '2019' AND EXTRACT('MONTH' FROM a.lpep_pickup_datetime)='09'
AND b."Zone"='Astoria'
ORDER BY a.tip_amount DESC
LIMIT 1;""", con=engine)

,Zone
0,JFK Airport


## Terraform

In this section homework we'll prepare the environment by creating resources in GCP with Terraform.

In your VM on GCP/Laptop/GitHub Codespace install Terraform. Copy the files from the course repo here to your VM/Laptop/GitHub Codespace.

Modify the files as necessary to create a GCP Bucket and Big Query Dataset.

Question 7. Creating Resources

After updating the main.tf and variable.tf files run:
```bash
terraform apply
```
Paste the output of this command into the homework submission form.

```bash
Terraform used the selected providers to generate the following execution plan. Resource actions
are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # google_bigquery_dataset.demo_dataset will be created
  + resource "google_bigquery_dataset" "demo_dataset" {
      + creation_time              = (known after apply)
      + dataset_id                 = "demo_dataset"
      + default_collation          = (known after apply)
      + delete_contents_on_destroy = false
      + effective_labels           = (known after apply)
      + etag                       = (known after apply)
      + id                         = (known after apply)
      + is_case_insensitive        = (known after apply)
      + last_modified_time         = (known after apply)
      + location                   = "EU"
      + max_time_travel_hours      = (known after apply)
      + project                    = "project_id"
      + self_link                  = (known after apply)
      + storage_billing_model      = (known after apply)
      + terraform_labels           = (known after apply)
    }

  # google_storage_bucket.demo-bucket will be created
  + resource "google_storage_bucket" "demo-bucket" {
      + effective_labels            = (known after apply)
      + force_destroy               = true
      + id                          = (known after apply)
      + location                    = "EU"
      + name                        = "demo-bucket"
      + project                     = (known after apply)
      + public_access_prevention    = (known after apply)
      + rpo                         = (known after apply)
      + self_link                   = (known after apply)
      + storage_class               = "STANDARD"
      + terraform_labels            = (known after apply)
      + uniform_bucket_level_access = (known after apply)
      + url                         = (known after apply)

      + lifecycle_rule {
          + action {
              + type = "AbortIncompleteMultipartUpload"
            }
          + condition {
              + age                   = 1
              + matches_prefix        = []
              + matches_storage_class = []
              + matches_suffix        = []
              + with_state            = (known after apply)
            }
        }
    }

Plan: 2 to add, 0 to change, 0 to destroy.

Do you want to perform these actions?
  Terraform will perform the actions described above.
  Only 'yes' will be accepted to approve.

  Enter a value: yes

google_bigquery_dataset.demo_dataset: Creating...
google_storage_bucket.demo-bucket: Creating...
google_storage_bucket.demo-bucket: Creation complete after 2s [id=...]
google_bigquery_dataset.demo_dataset: Creation complete after 2s [id=...]
```